In [44]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from numpy import array, savetxt
import numpy as np
import pandas as pd
import PIL
import glob
from PIL import Image
import cv2
import sys
import os
import csv
from os import listdir
import matplotlib.pyplot as plt

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center: #comparison function
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
      
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    
    power_val = [128,64,32,16,8,4,2,1]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]

    return val 
    
def lbp(img,height,width):
    img2=np.pad(img,pad_width=1) #padding the initial matrix
    height=img2.shape[0]
    width=img2.shape[1]
    img_lbp=np.zeros((height-2, width-2), dtype=int)

    for i in range(0, height-2):
        for j in range(0, width-2):
            img_lbp[i][j] = lbp_calculated_pixel(img2, i+1, j+1) #calling the necessary functions for lbp calculation and removing the padding
                
    
    return img_lbp

def mblbp(img,x,y):
    img1=[]
    avg=0
    
    for i in range(0,x-1,2):
        for j in range(0,y-1,2):
            avg=(int)((img[i][j]+img[i][j+1]+img[i+1][j]+img[i+1][j+1])/4) #calculating the average of 2x2 matrix 
            img1.append(avg)
    
    img1=np.array(img1)
    x=(int)(x/2)
    y=(int)(y/2)
    img1=img1.reshape(x,y)      
    return img1

def calc(img,height,width):
    if(height>28 and width>28):
        img=lbp(img,height,width) #sending img for lbp calculation
        img=mblbp(img,height,width) #sending img to the mblbp function 
        height=(int)(height/2) #reducing the size by 2
        width=(int)(width/2)
        return calc(img,height,width) #calling the function recursively
    
    return img
    
def store_csv(img):
    with open('/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/train.csv', 'a') as f: 
        #writer= csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer = csv.writer(f)  
        writer.writerow(img) 
        

def main():
    img_dir_happy_train='/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/training /happy'
    img_dir_sad+train='/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/training /sad'
    
    img_dir_happy_test='/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/testing/happy'
    img_dir_sad_test='/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/testing/sad'
    
    
    data_path = os.path.join(img_dir_happy,'*g') 
    files = glob.glob(data_path) 
    data_happy = [] 
    for f1 in files: 
        img = cv2.imread(f1,0) 
        img = cv2.resize(img,(56,56))
        data_happy.append(img) 

    data_happy=np.array(data_happy)
        
    data_path = os.path.join(img_dir_sad,'*g') 
    files = glob.glob(data_path) 
    data_sad = [] 
    for f1 in files: 
        img = cv2.imread(f1,0)
        img = cv2.resize(img,(56,56))
        data_sad.append(img) 

    data_sad=np.array(data_sad)
    
    height=data_happy.shape[1]
    width=data_happy.shape[2]

    f = open("/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/train.csv", "w")
    f.truncate()
    f.close()
    
    rows=[]
    size=28*28
    with open('/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/train.csv', 'a') as f:
        f = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i in range(size):
            rows.append(f'pixels{i}')   
        f.writerow(rows)
        
    for i in range(0,10):
        img_lbp=calc(data_happy[i],height,width)
        img_lbp=list(img_lbp.flatten())
        store_csv(img_lbp)
        img_lbp=calc(data_sad[i],height,width)
        img_lbp=list(img_lbp.flatten())
        store_csv(img_lbp)
        
    
                

if __name__ == '__main__':
    main()

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3478113043.py, line 97)